# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [2]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [3]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [5]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train = X_train.copy() # salvo una copia dei dati originali
    X_val = X_val.copy()
    
    if hyperparams['use_pca']:
        #...
        if hyperparams['pca_standardize']: #controllo se bisogna standardizzare nella PCA
            scaler_pca = StandardScaler()
            X_train = scaler_pca.fit_transform(X_train)
            X_val = scaler_pca.transform(X_val)
        
        pca = PCA(n_components= hyperparams['pca_components']) 
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val) 


    if hyperparams['data_standardize']:
        #...
        scaler_data = StandardScaler()
        X_train = scaler_data.fit_transform(X_train)
        X_val = scaler_data.transform(X_val)
        

    # Aggiunge il termine costante ai dati
    # ...
    vettore_costante_train = np.ones (X_train.shape[0])
    vettore_costante_validation = np.ones (X_val.shape[0])

    train_x =np.c_[vettore_costante_train, X_train]
    validation_x =np.c_[vettore_costante_validation, X_val]

    #print(train_x.shape)
    #print(validation_x.shape)

    
    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        reg_lambda = hyperparams['reg_lambda']
        I = np.eye(train_x.shape[1])
        I[-1, -1] = 0  # non regolarizzo il bias
        X_train_trasposta=train_x.T
        theta =np.linalg.inv(X_train_trasposta@train_x + reg_lambda * I )@X_train_trasposta@y_train # formula regressione lineare
        
    else:
        X_train_trasposta=train_x.T
        theta = np.linalg.inv(X_train_trasposta@train_x)@X_train_trasposta@y_train


    # Calcolo predizioni
    y_train_predizioni = train_x @ theta
    y_val_predizioni = validation_x @ theta

    #print (y_train_predizioni.shape)
    #print (y_val_predizioni.shape)

    
    # Calcola il MAE
    # ...
    MAE_train= np.mean(np.abs(y_train - y_train_predizioni))
    MAE_validation=np.mean(np.abs(y_val - y_val_predizioni))

    return MAE_train, MAE_validation, theta
    

# Dividi il dataset in training e test set
# ...

train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train_full = X[:num_train]
y_train_full = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]
    

# Dividi il training set in training set effettivo e validation set
# ...
num_train_effettivo = int(train_fraction * X_train_full.shape[0])

X_train = X_train_full[:num_train_effettivo]
y_train = y_train_full[:num_train_effettivo]

X_validation = X_train_full[num_train_effettivo:]
y_validation = y_train_full[num_train_effettivo:]


# Trova la configurazione di iperparametri migliore
# ...
best_MAE_val = float('inf')
best_config = None
best_parametri = None

for config in configs:
    MAE_train,MAE_val,_ = pipeline(X_train, y_train, X_validation, y_validation, config)
    
    if MAE_val < best_MAE_val:
        best_MAE_val = MAE_val
        best_config = config


# Riallena il modello sul training set completo
# ...
MAE_train_full, _, theta_finale = pipeline(X_train_full, y_train_full, X_test, y_test, best_config)


# Calcola il MAE sul test set
# ...
X_test_transformed = X_test.copy() 
X_train_transformed = X_train_full.copy()

# Applica PCA se necessario
if best_config['use_pca']:
    if best_config['pca_standardize']:
        scaler_pca = StandardScaler()
        X_train_transformed = scaler_pca.fit_transform(X_train_transformed)
        X_test_transformed = scaler_pca.transform(X_test_transformed)
        
    pca = PCA(n_components=best_config['pca_components'])
    X_train_transformed = pca.fit_transform(X_train_transformed)
    X_test_transformed = pca.transform(X_test_transformed)

# Applica standardizzazione se necessario
if best_config['data_standardize']:
    scaler_data = StandardScaler()
    X_train_transformed = scaler_data.fit_transform(X_train_transformed)
    X_test_transformed = scaler_data.transform(X_test_transformed)

# Aggiunge la feature costante (colonna di 1) alla matrice delle features di test
vettore_costante_test = np.ones(X_test_transformed.shape[0])
test_x = np.c_[vettore_costante_test, X_test_transformed]

# Predizioni e MAE
y_test_predizioni = test_x @theta_finale
MAE_test = np.mean(np.abs(y_test - y_test_predizioni))

# Stampa risultati
# ...
print("Migliore configurazione:", best_config)
print(f"MAE Test: {MAE_test}")
print(f"MAE Train: {MAE_train_full}")

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Migliore configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE Test: 0.5456050721527609
MAE Train: 0.5984709283825108
